In [51]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import urllib.request as urllib
from selenium import webdriver



In [52]:
urllist = ["https://www.kroger.com/pl/laundry-detergents/26007?page=1",
           "https://www.kroger.com/pl/laundry-detergents/26007?page=2",
           "https://www.kroger.com/pl/laundry-detergents/26007?page=3",
           "https://www.kroger.com/pl/laundry-detergents/26007?page=4",
           "https://www.kroger.com/pl/laundry-detergents/26007?page=5",
           "https://www.kroger.com/pl/laundry-detergents/26007?page=6"]

In [53]:
def get_prodInfo(allproducts):
    prodInfo={}
    df=pd.DataFrame()
    for tag in allproducts:
        prodInfo['title']= tag.h3.text
        for a in tag.find_all('img'):
            prodInfo['href'] = a['src']
            prodInfo['UPC'] = ''.join(filter(str.isdigit, a['src']))
        for price in tag.find_all('span',{'class':'kds-Price-singular'}):
            prodInfo['price'] = price.text
        for unit in tag.find_all('div',{'class':'ProductCard-sellBy ProductCard-sellBy-unit'}):
            prodInfo['unit'] = unit.text
        df =df.append(prodInfo, ignore_index=True)
    return df

In [54]:
df_prod=pd.DataFrame()
for url in urllist:
    browser = webdriver.Chrome() #replace with .Firefox(), or with the browser of your choice
    browser.get(url) #navigate to the page
    innerHTML = browser.execute_script("return document.body.innerHTML")
    html = BeautifulSoup( innerHTML , "html" )
    allproducts = html.findAll('div',{'class':'AutoGrid-cell min-w-0'})
    df=get_prodInfo(allproducts)
    df_prod=df_prod.append(df, ignore_index=True)

In [55]:
df_prod

,UPC,href,price,title,unit
0,0001111087903,https://www.kroger.com/product/images/medium/f...,$8.99,Simple Truth™ Sweet Olive Blossom Liquid Laund...,100 fl oz
1,0001111087905,https://www.kroger.com/product/images/medium/f...,$8.99,Simple Truth™ Free & Clear Liquid Laundry Dete...,100 fl oz
2,0080812414531,https://www.kroger.com/product/images/medium/f...,$15.99,Mrs. Meyer's Clean Day Laundry Detergent Laven...,64 fl oz
3,0003700053437,https://www.kroger.com/product/images/medium/f...,$22.99,Tide Spring Meadow Laundry Detergent Power Pods,48 ct
4,0002420001134,https://www.kroger.com/product/images/medium/f...,$7.99,Purex Dirt Lift Action Natural Elements Linen ...,150 fl oz
...,...,...,...,...,...
156,0003700091798,https://www.kroger.com/product/images/medium/f...,$21.99,Tide PODS Free & Gentle Laundry Detergent Pacs...,81 ct
157,0003700093045,https://www.kroger.com/product/images/medium/f...,$21.99,Tide PODS Original 3 in 1 Liquid Laundry Deter...,81 ct
158,0007261346139,https://www.kroger.com/product/images/medium/f...,$21.99,All with Stainlifters Free Clear Liquid Laundr...,162 fl oz
159,0007261373945,https://www.kroger.com/product/images/medium/f...,$8.99,All with Stainlifters Free Clear Liquid Laundr...,88 fl oz


In [57]:
df.to_csv('laundry-detergents.csv',index=False)